In [33]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [34]:
df = pd.read_csv('tyre_sales_raw_10000.csv')

In [35]:
df.head()

,Sale_ID,Date,Region,Tyre_Type,Tyre_Size,Units_Sold,Unit_Price,Total_Sales
0,S00001,2024-04-12,South,Biased,18,147.0,4408,647976.0
1,S00002,2024-06-28,East,Biased,15,90.0,5160,464400.0
2,S00003,2024-04-02,North,Biased,18,113.0,4801,542513.0
3,S00004,2024-01-15,North,Radial,14,104.0,3784,393536.0
4,S00005,2024-04-16,Eest,Biased,16,139.0,5033,699587.0


In [36]:
df.info()
df.isna().sum().sort_values(ascending=False).head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sale_ID      10000 non-null  object 
 1   Date         10000 non-null  object 
 2   Region       9700 non-null   object 
 3   Tyre_Type    10000 non-null  object 
 4   Tyre_Size    10000 non-null  int64  
 5   Units_Sold   9707 non-null   object 
 6   Unit_Price   10000 non-null  int64  
 7   Total_Sales  9700 non-null   float64
dtypes: float64(1), int64(2), object(5)
memory usage: 625.1+ KB


,0
Region,300
Total_Sales,300
Units_Sold,293
Sale_ID,0
Tyre_Type,0
Date,0
Tyre_Size,0
Unit_Price,0


In [37]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns


Index(['sale_id', 'date', 'region', 'tyre_type', 'tyre_size', 'units_sold',
       'unit_price', 'total_sales'],
      dtype='object')

In [38]:
df = df.drop_duplicates()
df.shape


(10000, 8)

In [39]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])
df.head()


,sale_id,date,region,tyre_type,tyre_size,units_sold,unit_price,total_sales
0,S00001,2024-04-12,South,Biased,18,147.0,4408,647976.0
1,S00002,2024-06-28,East,Biased,15,90.0,5160,464400.0
2,S00003,2024-04-02,North,Biased,18,113.0,4801,542513.0
3,S00004,2024-01-15,North,Radial,14,104.0,3784,393536.0
4,S00005,2024-04-16,Eest,Biased,16,139.0,5033,699587.0


In [40]:
for col in ['units_sold', 'unit_price']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [41]:
df[['units_sold', 'unit_price']].head()


,units_sold,unit_price
0,147.0,4408
1,90.0,5160
2,113.0,4801
3,104.0,3784
4,139.0,5033


In [42]:
df = df.dropna(subset=['units_sold', 'unit_price'])
df.shape


(9125, 8)

In [43]:
if 'region' in df.columns:
    df['region'] = df['region'].fillna('Unknown')

if 'tyre_type' in df.columns:
    df['tyre_type'] = df['tyre_type'].fillna('Unknown')


In [44]:
text_cols = ['region', 'tyre_type']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()
df[text_cols].head()


,region,tyre_type
0,South,Biased
1,East,Biased
2,North,Biased
3,North,Radial
4,Eest,Biased


In [45]:
q1 = df['units_sold'].quantile(0.25)
q3 = df['units_sold'].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5*iqr
upper = q3 + 1.5*iqr

df['units_sold'] = df['units_sold'].clip(lower, upper)
df['units_sold'].describe()


,units_sold
count,9125.000000
mean,99.827507
std,28.718533
min,50.000000
25%,75.000000
50%,100.000000
75%,125.000000
max,149.000000


In [46]:
df['revenue'] = df['units_sold'] * df['unit_price']
df[['units_sold','unit_price','revenue']].head()


,units_sold,unit_price,revenue
0,147.0,4408,647976.0
1,90.0,5160,464400.0
2,113.0,4801,542513.0
3,104.0,3784,393536.0
4,139.0,5033,699587.0


In [47]:
df.to_csv('/content/tyre_sales_clean.csv', index=False)

In [48]:
from google.colab import files
files.download('/content/tyre_sales_clean.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>